In [1]:
import pandas as pd
import numpy as np

In [2]:
data_scrap = pd.read_json('raw_data/scrapped_dealls.json')
data_scrap['jobs_id'] = data_scrap.index

In [3]:
data_scrap.drop(['job_desc_list', 'job_qualification_list'], axis=1, inplace=True)

# Cleaning Data

## Company Employees Count

In [4]:
data_scrap['company_employess_count'] = data_scrap['company_employess_count'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
data_scrap['company_employess_count'] = data_scrap['company_employess_count'].replace(['–', '-', '—'], np.nan)
data_scrap['company_employess_count'] = data_scrap['company_employess_count'].fillna('-')
data_scrap['company_employess_count'].value_counts()

company_employess_count
>100      1300
50–100     839
-          811
1–50       246
Name: count, dtype: int64

## Company Location

In [5]:
data_scrap['company_location'] = data_scrap['company_location'].apply(lambda x: x[0].strip() if len(x) > 0 else np.nan)
data_scrap['company_location'].value_counts()

company_location
Jakarta Selatan      1398
Jakarta               628
Jakarta Barat         229
Jakarta Utara         138
Jakarta Pusat         137
Jakarta Timur         136
Tangerang Selatan      76
Tangerang              72
Yogyakarta             67
Sidoarjo Regency       58
Bandung                44
Bekasi                 42
Banten                 33
Surabaya               29
Indonesia              26
Tangerang Regency      26
Sleman Regency         13
Salatiga                8
Badung Regency          6
Depok                   5
Bogor                   4
Semarang Regency        3
Bogor Regency           2
Karawang Regency        2
Bali                    2
Batam                   2
Sudbury, Ontario        2
Pasuruan                1
Bekasi Regency          1
Kuningan Regency        1
Singapore               1
Semarang                1
Tebing Tinggi           1
Malang Regency          1
Cikarang                1
Name: count, dtype: int64

## Min Experience

In [6]:
data_scrap['min_experience'] = np.where(
        data_scrap['min_experience'] == 'Open for all levels',
        'Open for all levels',
        data_scrap['min_experience'].apply(
            lambda x: x if str(x).startswith('Min.') else f'Min. {x}'
        )
    )

data_scrap['min_experience'].value_counts()

min_experience
Min. 1 years of experience       1573
Min. Freshgrad                    787
Min. 4 years of experience        345
Min. 4th Year College Student     281
Min. 5+ years of experience       114
Min. 3rd Year College Student      50
Min. 1st Year College Student      35
Min. 2nd Year College Student       9
Open for all levels                 2
Name: count, dtype: int64

In [7]:
data_scrap['min_experience'].isna().sum()

0

## Company Industry

In [8]:
# Jika ternyata list, gunakan ini
def extract_from_list(item):
    if isinstance(item, list):
        return item[0] if item else ""
    if isinstance(item, str):
        return item.strip('[]')
    return item

# Coba kedua pendekatan
data_scrap['company_industry'] = data_scrap['company_industry'].apply(extract_from_list)
data_scrap['company_industry'].head()

0    Technology and Software Solutions
1                    Fashion & Apparel
2                 Beauty and Cosmetics
3                    Fashion & Apparel
4       Human Resources and Consulting
Name: company_industry, dtype: object

In [9]:
data_scrap['company_industry'].isna().sum()

0

## Company Desc

In [10]:
data_scrap['company_desc'] = data_scrap['company_desc'].apply(extract_from_list)
data_scrap['company_desc'].head()

0    We are an Integrated IT Solutions provider wit...
1    Torch.id is a prominent Indonesian brand, prov...
2    Clerina Group merupakan company yang bergerak ...
3    Torch.id is a prominent Indonesian brand, prov...
4    PT KHARISMA POTENSIA INDONESIA Committed to be...
Name: company_desc, dtype: object

In [11]:
data_scrap['company_desc'].isna().sum()

0

## Job Desc, Job Req

In [12]:
cleaned_job_desc_req = pd.read_csv('raw_data/cleaned_job_desc_qualification.csv')
cleaned_job_desc_req.head()

,jobs_id,job_desc_list_len,job_desc_list,job_qualification_list_len,job_qualification_list
0,2223,514,['Responsible for supporting Project Manager f...,864,"[""A Bachelor's Degree in Business, Project Man..."
1,1646,400,['To analyze and validate data source readines...,418,"[""A Bachelor's Degree in Computer Science, Inf..."
2,3172,327,"['Analyze, design, and develop a new feature',...",558,"[""A bachelor's degree in Information Technolog..."
3,110,358,"[""As a Crossborder Executive (Amazon), you wil...",586,"[""A bachelor's degree in marketing, e-commerce..."
4,1513,886,['Participate in formulating the long-term str...,480,"[""A bachelor's degree or above, with a prefere..."


In [13]:
# def clean_job_text(text_list):
#     """
#     Cleans job descriptions and qualifications by processing each item in a list of strings
#     to remove patterns like numbered bullets, special characters, and other non-letter 
#     beginnings. Ensures the first character is a letter.
    
#     Args:
#         text_list: List of strings or a single string
        
#     Returns:
#         List of cleaned strings or a cleaned string
#     """
#     import re
    
#     # If input is not a list or is NaN, return as is
#     if not isinstance(text_list, list):
#         return text_list
    
#     cleaned_list = []
    
#     for item in text_list:
#         if not isinstance(item, str):
#             cleaned_list.append(item)
#             continue
            
#         # Remove numbered bullets (e.g., "1. ", "2. ", "10. ")
#         item = re.sub(r'^\d+\.\s+', '', item)
        
#         # Remove Chinese-style numbering (e.g., "1、")
#         item = re.sub(r'^\d+、\s*', '', item)
        
#         # Remove colons at the beginning (e.g., ": Text")
#         item = re.sub(r'^:\s+', '', item)
        
#         # Remove periods at the beginning (e.g., ".Text")
#         item = re.sub(r'^\.', '', item)
        
#         # Remove Unicode WORD JOINER character (\u2060)
#         item = item.replace('\u2060', '')
        
#         # Remove special symbols at the beginning
#         item = re.sub(r'^[📌♦\·•\-_\+=>#@\$%\^&~\|]+\s', '', item)
        
#         # Remove leading spaces and invisible characters
#         item = item.lstrip()
        
#         # Ensure the string starts with a letter - if not, keep removing characters until it does
#         while item and not item[0].isalpha():
#             item = item[1:] if len(item) > 1 else ""
        
#         cleaned_list.append(item)
    
#     return cleaned_list


In [14]:
data_scrap = data_scrap.merge(cleaned_job_desc_req, on='jobs_id', how='right').sort_values('jobs_id').drop(['job_desc_list_len', 'job_qualification_list_len'], axis=1)
data_scrap.head()

,url,company_profile_src,company_name,job_position,employment_type,working_location_type,working_location,min_experience,salary,company_desc,company_industry,company_location,company_employess_count,jobs_id,job_desc_list,job_qualification_list
1594,https://dealls.com/loker/front-end-developer-3...,https://cdn.sejutacita.id/64f7ee28f62e160013a6...,Bringin Inti Teknologi,Back End Developer,Penuh waktu,On-site,Jakarta,Min. 1 years of experience,Negotiable,We are an Integrated IT Solutions provider wit...,Technology and Software Solutions,Jakarta Pusat,50–100,0,"['Utilize languages like Go, PHP/Laravel, and ...",['Minimum Bachelor of Science in computer scie...
1970,https://dealls.com/loker/freelance-sales-retai...,https://cdn.sejutacita.id/62fba7bb7cc7a000147e...,Torch.id,(Freelance) Sales Retail Mobile/Canvasser,Penuh waktu,On-site,Indonesia,Min. 1 years of experience,Negotiable,"Torch.id is a prominent Indonesian brand, prov...",Fashion & Apparel,Bandung,50–100,1,['Retail Sales Mobile akan bertanggung jawab u...,"['Pendidikan minimal SMA/SMK/sederajat.', 'Mem..."
1877,https://dealls.com/loker/creative-lead-2~aurah...,https://cdn.sejutacita.id/66b46d83438f440013dd...,Clerina Group,Creative Lead,Penuh waktu,On-site,Bandung,Min. 1 years of experience,Negotiable,Clerina Group merupakan company yang bergerak ...,Beauty and Cosmetics,Bandung,1–50,2,"['Membuat laporan weekly & monthly', 'Memberik...",['Pendidikan minimal S1 Ilmu Komunikasi / Mark...
130,https://dealls.com/loker/social-media-speciali...,https://cdn.sejutacita.id/62fba7bb7cc7a000147e...,Torch.id,Social Media Specialist,Penuh waktu,On-site,Bandung,Min. 1 years of experience,Negotiable,"Torch.id is a prominent Indonesian brand, prov...",Fashion & Apparel,Bandung,50–100,3,['We are looking for a Social Media Specialist...,"[""Bachelor's degree in Marketing, Communicatio..."
2145,https://dealls.com/loker/hrga-staff-area-padal...,https://cdn.sejutacita.id/665fd307c547c5001327...,Kharisma Potensia Indonesia,HRGA Staff area Padalarang Kabupaten Bandung B...,Kontrak,On-site,Bandung Barat Regency,Min. 1 years of experience,Rp3.500.000 – 3.508.677,PT KHARISMA POTENSIA INDONESIA Committed to be...,Human Resources and Consulting,Jakarta,>100,4,['Membantu perekrutan dan pengelolaan operasio...,"['Pengalaman HRGA minimal 1 Tahun', 'Memiliki ..."


In [15]:
def clean_special_chars(text):
    """
    Cleans text by replacing specific special characters:
    1. \xa0 (non-breaking space) -> regular space " "
    2. â€™ (curly apostrophe encoding issue) -> '
    
    Args:
        text: String to clean
        
    Returns:
        Cleaned string
    """
    if isinstance(text, str):
        # Replace \xa0 with space
        text = text.replace('\xa0', ' ')
        
        # Replace â€™ with apostrophe
        text = text.replace('â€™', "'")
        
        return text
    return text

# Contoh penggunaan:
data_scrap['job_desc_list'] = data_scrap['job_desc_list'].apply(clean_special_chars)
data_scrap['job_qualification_list'] = data_scrap['job_qualification_list'].apply(clean_special_chars)

In [16]:
data_scrap['working_location'] = data_scrap['working_location'].fillna('-')
data_scrap.isna().sum()

url                        0
company_profile_src        0
company_name               0
job_position               0
employment_type            0
working_location_type      0
working_location           0
min_experience             0
salary                     0
company_desc               0
company_industry           0
company_location           0
company_employess_count    0
jobs_id                    0
job_desc_list              0
job_qualification_list     0
dtype: int64

## Merge with Category

In [50]:
data_ori_awal = pd.read_json('raw_data/scrapped_dealls.json')
data_ori_awal['jobs_id'] = data_ori_awal.index
jobs_company_category_df = data_ori_awal.merge(pd.read_csv('raw_data/multi_label_category_gemini_pro.csv')[['jobs_id', 'kategori']] , on='jobs_id')[['job_position', 'company_name', 'kategori']]

In [57]:
data_ori_awal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   url                      3196 non-null   object
 1   company_profile_src      3196 non-null   object
 2   company_name             3196 non-null   object
 3   job_position             3196 non-null   object
 4   employment_type          3196 non-null   object
 5   working_location_type    3196 non-null   object
 6   working_location         3154 non-null   object
 7   min_experience           3196 non-null   object
 8   salary                   3196 non-null   object
 9   job_desc_list            3196 non-null   object
 10  job_qualification_list   3196 non-null   object
 11  company_desc             3196 non-null   object
 12  company_industry         3196 non-null   object
 13  company_location         3196 non-null   object
 14  company_employess_count  3196 non-null  

DATA SCRAP ADALAH DATA YANG BERSIH DARI EXCEL.

In [63]:
data_scrap_with_labeled_gemini = data_scrap.merge(jobs_company_category_df, how='left', on=['job_position', 'company_name'])
data_scrap_with_labeled_gemini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725 entries, 0 to 2724
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   url                      2725 non-null   object
 1   company_profile_src      2725 non-null   object
 2   company_name             2725 non-null   object
 3   job_position             2725 non-null   object
 4   employment_type          2725 non-null   object
 5   working_location_type    2725 non-null   object
 6   working_location         2725 non-null   object
 7   min_experience           2725 non-null   object
 8   salary                   2725 non-null   object
 9   company_desc             2725 non-null   object
 10  company_industry         2725 non-null   object
 11  company_location         2725 non-null   object
 12  company_employess_count  2725 non-null   object
 13  jobs_id                  2725 non-null   int64 
 14  job_desc_list            2725 non-null  

In [64]:
cleaned_dataset_without_desc = data_scrap_with_labeled_gemini.drop(['job_desc_list', 'job_qualification_list', 'url', 'company_profile_src', 'company_desc'], axis=1)
cleaned_dataset_without_desc

,company_name,job_position,employment_type,working_location_type,working_location,min_experience,salary,company_industry,company_location,company_employess_count,jobs_id,kategori
0,Bringin Inti Teknologi,Back End Developer,Penuh waktu,On-site,Jakarta,Min. 1 years of experience,Negotiable,Technology and Software Solutions,Jakarta Pusat,50–100,0,['Software Engineering']
1,Torch.id,(Freelance) Sales Retail Mobile/Canvasser,Penuh waktu,On-site,Indonesia,Min. 1 years of experience,Negotiable,Fashion & Apparel,Bandung,50–100,1,"['Sales', 'Retail']"
2,Clerina Group,Creative Lead,Penuh waktu,On-site,Bandung,Min. 1 years of experience,Negotiable,Beauty and Cosmetics,Bandung,1–50,2,"['Creative & Design', 'Management & Leadership']"
3,Torch.id,Social Media Specialist,Penuh waktu,On-site,Bandung,Min. 1 years of experience,Negotiable,Fashion & Apparel,Bandung,50–100,3,"['Marketing', 'Creative & Design']"
4,Kharisma Potensia Indonesia,HRGA Staff area Padalarang Kabupaten Bandung B...,Kontrak,On-site,Bandung Barat Regency,Min. 1 years of experience,Rp3.500.000 – 3.508.677,Human Resources and Consulting,Jakarta,>100,4,"['Human Resources', 'Administrative & Secretar..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2720,BRIK,Batching Plant Leader,Kontrak,On-site,Indonesia,Min. 1 years of experience,Negotiable,Building Materials,Jakarta Barat,-,3191,"['Production & Manufacturing', 'Management & L..."
2721,Evolene Group,Content Creator Talent,Penuh waktu,On-site,Tangerang,Min. 1 years of experience,Rp5.000.000 – 7.000.000,Fitness,Tangerang,50–100,3192,"['Creative & Design', 'Marketing']"
2722,PT. Mitra Manunggal Mahardika (Adelle Jewellery),Online Sales Executive,Penuh waktu,On-site,Jakarta Utara,Min. 1 years of experience,Negotiable,Luxury Goods & Jewelry,Jakarta,>100,3193,"['Sales', 'E-commerce']"
2723,PT Indonesia Fintopia Technology (Easycash),Complaint Expert Specialist,Kontrak,On-site,Jakarta Selatan,Min. 4 years of experience,Negotiable,"Tech Company, Fintech",Jakarta,50–100,3194,"['Customer Service & Support', 'Risk Management']"


In [65]:
import json

# First convert to a list of dictionaries
records = cleaned_dataset_without_desc.to_dict(orient='records')

# Then use json.dump with ensure_ascii=False
with open('coba.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

In [66]:
cleaned_dataset_without_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725 entries, 0 to 2724
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   company_name             2725 non-null   object
 1   job_position             2725 non-null   object
 2   employment_type          2725 non-null   object
 3   working_location_type    2725 non-null   object
 4   working_location         2725 non-null   object
 5   min_experience           2725 non-null   object
 6   salary                   2725 non-null   object
 7   company_industry         2725 non-null   object
 8   company_location         2725 non-null   object
 9   company_employess_count  2725 non-null   object
 10  jobs_id                  2725 non-null   int64 
 11  kategori                 2725 non-null   object
dtypes: int64(1), object(11)
memory usage: 255.6+ KB


## Fill Salary

In [34]:
salary_label_gemini = pd.read_csv('raw_data/salary_flash.csv')
salary_label_gemini

,jobs_id,salary
0,0,Rp5.000.000 – 10.000.000
1,1,Rp3.000.000 – 5.000.000
2,2,Rp6.000.000 – 12.000.000
3,3,Rp4.000.000 – 8.000.000
4,6,Rp4.000.000 – 7.000.000
...,...,...
2192,3191,Rp5.000.000 – 10.000.000
2193,3192,Rp5.000.000 – 7.000.000
2194,3193,Rp4.000.000 – 8.000.000
2195,3194,Rp6.000.000 – 12.000.000


In [48]:
data_scrap_with_salary = data_scrap_with_labeled_gemini.merge(salary_label_gemini, how='left', on='jobs_id')
data_scrap_with_salary['salary_y'] = data_scrap_with_salary['salary_y'].fillna(data_scrap_with_salary['salary_x'])
data_scrap_with_salary['salary'] = data_scrap_with_salary['salary_y']
data_scrap_with_salary = data_scrap_with_salary.drop(['salary_x', 'salary_y'], axis=1)

In [49]:
data_scrap_with_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725 entries, 0 to 2724
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   url                      2725 non-null   object
 1   company_profile_src      2725 non-null   object
 2   company_name             2725 non-null   object
 3   job_position             2725 non-null   object
 4   employment_type          2725 non-null   object
 5   working_location_type    2725 non-null   object
 6   working_location         2725 non-null   object
 7   min_experience           2725 non-null   object
 8   company_desc             2725 non-null   object
 9   company_industry         2725 non-null   object
 10  company_location         2725 non-null   object
 11  company_employess_count  2725 non-null   object
 12  jobs_id                  2725 non-null   int64 
 13  job_desc_list            2725 non-null   object
 14  job_qualification_list   2725 non-null  

In [67]:
data_scrap_with_salary.to_csv('cleaned_data/merged_full_clean_data.csv', index=False)